In [ ]:
import requests

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [ ]:
data = requests.get(standings_url)


In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select('table.stats_table')[0]
print(standings_table)

In [ ]:
# assigning all the a tags in the html pages to variable of a links 
links = standings_table.find_all('a')


In [ ]:
links = [l.get("href" for l in links)]
links = [l for l in links if '/squad/' in l]

In [ ]:
team_urls = [f"https://fbref/com{l}" for l in links]